This notebook is used to calculate the simplified ductbank model in GridLAB-D based on a detailed model of a ductbank.

----

The ductbank model is the equilibrium thermal model (i.e., thermal mass is not considered) described by the following:

$$
    Q   \xrightarrow{} (T_C) 
        \xrightarrow{U_I} (T_S) 
        \xrightarrow{U_A} (T_I)
        \xrightarrow{U_D} (T_O)
        \xrightarrow{U_F} (T_G)
$$

where
- $Q$ is the thermal heat from the cable losses (input)
- $T_C$ is the temperature of the cable core
- $U_I$ is the thermal conductance of the cable insulation
- $T_S$ is the temperature of the cable shield
- $U_A$ is the thermal conductance of the duct air gap
- $T_I$ is the temperature of the duct inner surface
- $U_D$ is the thermal conductance of the duct core
- $T_O$ is the temperature of the duct outer surface
- $U_F$ is the thermal conductance of the fill soil
- $T_G$ is the temperature of the surface ground

![Example Box 3x2](box3x2.png "Example Box 3x2")

In [19]:
import sys
from numpy import pi, sqrt, array, set_printoptions

set_printoptions(formatter={'float_kind':"{:6.4g}".format})

INPUT = sys.stdin
OUTPUT = sys.stdout

def error(msg,code=None):
	print(f"ERROR [create_ductbank]: {msg} (code %d)",file=sys.stderr)
	if type(code) is int:
		exit(code)

class Ductbank:

    name = None
    surface_temperature = 25.0 # degC
    warning_cable_temperature = 80.0 # degC
    alert_cable_temperature = 90.0 # degC"
    duct_ROH = 60.0 # W.cm/K
    insulation_ROH = 120.0 # W.cm/K
    insulation_thickness = 1.0 # cm
    air_ROH = 20.0 # W.cm/K
    rows = 3
    columns = 2
    channel_diameter = 5 # in
    horizontal_spacing = 1.5 # in
    vertical_spacing = 0.5 # in
    side_thickness = 1.5 # in
    top_thickness = 1.5 # in
    bottom_thickness = 1.5 # in
    cable_diameter = 1 # in
    channel_loading = [[3,0],[0,3],[3,0]] # cable count in each duct
    channel_heating = [[1,0],[0,1],[1,0]] # W/m 

    def A(self,i,j):
        d = 0.0
        # vertical spacing
        if i == 0:
            d += self.top_thickness
        elif i < self.rows-1:
            d += self.vertical_spacing
        if i == self.rows-1:
            d += self.top_thickness
        elif i < self.rows-1:
            d += self.vertical_spacing
        # horizontal spacing
        if j == 0:
            d += self.side_thickness
        elif i < self.columns-1:
            d += self.horizontal_spacing
        if j == self.columns-1:
            d += self.side_thickness
        elif i < self.columns-1:
            d += self.horizontal_spacing
        Ud = d*self.duct_ROH/100 
        Ui = self.insulation_ROH*self.insulation_thickness
        A = (self.channel_diameter/2)**2 * pi
        C = (self.cable_diameter/2)**2 * pi * self.channel_loading[i][j]
        if C > A:
            raise f"channel {i},{j} is overloaded"
        Ua = sqrt(A-C)/2.54 * self.air_ROH
        return 1/(1/Ud+1/Ui+1/Ua)

    def B(self,i,k):
        if k == 0: # temperature term
            R = 0
            for j in range(self.columns):
                if i > 0: # not top row
                    if j == 0 or j == self.columns-1: # side (not sure whether 1 or both)
                        d = 0.0
                        if i == 0: # left side
                            d += self.side_thickness
                        if i == self.columns - 1: # right side
                            d += self.side_thickness
                        if d > 0:
                            R += 100/(d*self.duct_ROH)
                else: # top row
                    if j > 0 or j <self.columns-1: # not side duct
                        R += 100/(self.top_thickness*self.duct_ROH)
                    else: # side (not sure whether 1 or both)
                        d = 0.0
                        if i == 0: # left side
                            d += self.side_thickness
                        if i == self.columns - 1: # right side
                            d += self.side_thickness
                        if d > 0 :
                            R +=  100/(d*self.duct_ROH/100 + self.top_thickness*self.duct_ROH)
            return 1/R if R > 0 else 0.0
        else: # heat gain term
            return 1 if i+1 == k else 0            

    def u(self,i,j):
        return self.channel_loading[i][j]

In [24]:
ductbank = Ductbank()

In [10]:
from numpy.linalg import solve
print("object ductbank",file=OUTPUT)
print("{",file=OUTPUT)
for tag in dir(ductbank):
    if not tag.startswith("_"):
        value = getattr(ductbank,tag)
        if type(value) in [str,float,int,list] and value != None:
            print(f'    {tag} "{value}";',file=OUTPUT)
print("}",file=OUTPUT)

object ductbank
{
    air_ROH "20.0";
    alert_cable_temperature "90.0";
    bottom_thickness "1.5";
    cable_diameter "1";
    channel_diameter "5";
    channel_heating "[[1, 0], [0, 1], [1, 0]]";
    channel_loading "[[3, 0], [0, 3], [3, 0]]";
    columns "2";
    duct_ROH "60.0";
    horizontal_spacing "1.5";
    insulation_ROH "120.0";
    insulation_thickness "1.0";
    rows "3";
    side_thickness "1.5";
    surface_temperature "25.0";
    top_thickness "1.5";
    vertical_spacing "0.5";
    warning_cable_temperature "80.0";
}


In [11]:
K = ductbank.rows*ductbank.columns
A = []
for r in range(K):
    A.append([])
    for i in range(ductbank.rows):
        for j in range(ductbank.columns):
            A[r].append(ductbank.A(i,j))
A = array(A)
print("A =\n" + str(A))

A =
[[ 2.687    2.7  1.421  1.417  1.682  1.688]
 [ 2.687    2.7  1.421  1.417  1.682  1.688]
 [ 2.687    2.7  1.421  1.417  1.682  1.688]
 [ 2.687    2.7  1.421  1.417  1.682  1.688]
 [ 2.687    2.7  1.421  1.417  1.682  1.688]
 [ 2.687    2.7  1.421  1.417  1.682  1.688]]


In [17]:
B = []
for i in range(K):
    B.append([])
    for j in range(K+1):
        B[i].append(ductbank.B(i,j))
B = array(B)
print("B =\n" + str(B))

B =
[[  0.45      1      0      0      0      0      0]
 [  0.45      0      1      0      0      0      0]
 [     0      0      0      1      0      0      0]
 [     0      0      0      0      1      0      0]
 [     0      0      0      0      0      1      0]
 [     0      0      0      0      0      0      1]]


In [25]:
u = [ductbank.surface_temperature]
for i in range(ductbank.rows):
    for j in range(ductbank.columns):
        u.append(ductbank.u(i,j))
u = array([u]).transpose()
print("u =\n" + str(u))

u =
[[    25]
 [     3]
 [     0]
 [     0]
 [     3]
 [     3]
 [     0]]


In [26]:
solve(A,B@u)

LinAlgError: Singular matrix